In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import spatial
from datetime import timedelta
import seaborn as sns
import scipy.stats as stats
sns.set_style("white")
from gatspy.periodic import LombScargleFast
from gatspy.periodic import LombScargle
from datetime import timedelta

df = pd.read_csv('D:\data from MnM Samdal raw\Samdal_power_30min_no_outliers.csv',  parse_dates=[0])

df = df.rename( columns={"Unnamed: 0": "time"})


In [2]:
# no smoothing
df['new_speed'] = df.speed

df = df.dropna(subset=['new_speed'])

In [2]:
# with  smoothing

# resample to 30s
win_size0 = 4  # 2h avg, based on 30s interval data

df2sspeed = df.speed.rolling(win_size0, min_periods = int(win_size0)).mean()

# add nan of half size of the window at the beginning of df to better match the series
nn = df2sspeed[int(win_size0/2):].values
tap = np.nan * np.empty(int(win_size0/2))

mergedlist = []
mergedlist.extend(nn)
mergedlist.extend(tap)
df['new_speed'] = mergedlist

df = df.dropna(subset=['new_speed'])

In [65]:
plt.plot(df.new_speed, 'o')
plt.plot(df.speed, c='r')

plt.show()

In [3]:
t = df.time
#mag = df.speed
mag = df.new_speed

# calc differences from the beginning of time to create a new time index
subtract = lambda x: x - t.iloc[0]  # iloc because [] takes by index value, and iloc takes first value from the df
df['time_diff'] = t.apply(subtract)

In [4]:
df= df.set_index('time')

In [5]:
# only for resol. bigger than sec

# count time from the beginning as a number, not as a datetime

#to_min = lambda x: (x.total_seconds())/1800  # to 30min 
to_min = lambda x: (x.total_seconds())/86400  # to 1 day

df['time_diff_conv'] = df.time_diff.apply(to_min)   # to a number
t= df['time_diff_conv']


In [7]:
# model optimization, looking for the right period

model = LombScargleFast(fit_period=True)

model.optimizer.period_range = (0.1, 90) # range should be in the same units as data
model.fit(t, mag)
model.best_period


Finding optimal frequency:
 - Estimated peak width = 0.0362
 - Using 5 steps per peak; omega_step = 0.00725
 - User-specified period range:  0.1 to 90
 - Computing periods at 8663 steps
Zooming-in on 5 candidate peaks:
 - Computing periods at 1000 steps


83.419444493933611

In [9]:
# further model optimization, looking for the right period

model = LombScargleFast(fit_period=True)

model.optimizer.set(period_range=(81, 85), first_pass_coverage=10)
#model.optimizer.set(period_range=(0.9, 1.2), first_pass_coverage=10)

 # range should be in the same units as data
model.fit(t, mag)
model.best_period


Finding optimal frequency:
 - Estimated peak width = 0.0362
 - Using 100 steps per peak; omega_step = 0.000362
 - User-specified period range:  81 to 85
 - Computing periods at 12 steps
Zooming-in on 5 candidate peaks:
 - Computing periods at 50 steps


84.668000536100635

In [11]:
# minutes

model = LombScargleFast().fit(t, mag)
#model = LombScargle().fit(t, mag)

#periods, power = model.periodogram_auto(nyquist_factor=100)
periods, power = model.periodogram_auto()

fig, ax = plt.subplots()
ax.plot(periods, power)

plt.xticks(fontsize=22)
plt.yticks(fontsize=22)
ax.set_xlabel('period [30 min.]', fontsize=26)

ax.set_ylabel('Lomb-Scargle Power', fontsize=26)

plt.show()

